In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

# Scrap

El formato de nominatim es el siguiente

```json
[
    {
        "place_id": 239916922,
        "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
        "osm_type": "relation",
        "osm_id": 2110264,
        "lat": "30.6598628",
        "lon": "104.0633717",
        "category": "boundary",
        "type": "administrative",
        "place_rank": 10,
        "importance": 0.50001,
        "addresstype": "city",
        "name": "Chengdu",
        "display_name": "Chengdu, Sichuan, China",
        "boundingbox": [
            "30.0915500",
            "31.4370968",
            "102.9896230",
            "104.8948475"
        ]
    },
    {
        "place_id": 239270537,
        "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
        "osm_type": "node",
        "osm_id": 3899571938,
        "lat": "30.7648688",
        "lon": "102.1192947",
        "category": "place",
        "type": "village",
        "place_rank": 19,
        "importance": 0.27501,
        "addresstype": "village",
        "name": "Chengdu",
        "display_name": "Chengdu, Xiaojin County, Ngawa Tibetan and Qiang Autonomous Prefecture, Sichuan, China",
        "boundingbox": [
            "30.7448688",
            "30.7848688",
            "102.0992947",
            "102.1392947"
        ]
    }
]
```

In [3]:
response = requests.get("https://populationstat.com/china/")

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
tab = soup.find("table", id="ph4")

In [6]:
header = [ i.text for i in tab.find("thead").find_all("th")]
header = header[:2] + ["Province", "Latitude", "Longitude", "geometry"] + header[2:]

In [8]:
headers = {
    'authority': 'nominatim.openstreetmap.org',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Brave";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'sec-gpc': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
}

ans = []
for row in tab.find("tbody").find_all("tr"):
    temp = []
    for i,j in enumerate(row.find_all("td")):
        if i == 0 or i == 2 or i == 3:
            try:
                temp.append(int(j.text.replace(",", "")))
            except:
                temp.append(None)
        else:
            temp.append(j.text)
            try:
                city_url = f"https://nominatim.openstreetmap.org/search.php?city={j.text}&country=China&format=jsonv2"
                r = requests.get(city_url, headers=headers)
                #time.sleep(0.5)
                lat, lon = float(r.json()[0]["lat"]), float(r.json()[0]["lon"])
                centre_point = Point(lon, lat)
            except Exception as e:
                print(f"Error with {j.text}: {e}")
                print(r)
                lat, lon, centre_point = None, None, None
            try:
                reverse_url = f"https://nominatim.openstreetmap.org/reverse.php?lat={lat}&lon={lon}&zoom=5&format=jsonv2"
                r = requests.get(reverse_url, headers=headers)
                #time.sleep(0.5)
                province = r.json()["name"]
            except:
                province = None
            temp.append(province)
            temp.append(lat)
            temp.append(lon)
            temp.append(centre_point)
    print(temp)

    ans.append(temp)


[1, 'Shanghai', 'Shanghai', 31.2323437, 121.4691024, <POINT (121.469 31.232)>, 28242646, 22315474]
[2, 'Beijing', 'Beijing', 39.9057136, 116.3912972, <POINT (116.391 39.906)>, 21167094, 18960744]
[3, 'Chongqing', 'Chongqing', 29.5656729, 106.5479189, <POINT (106.548 29.566)>, 16687404, 7457599]
[4, 'Tianjin', 'Tianjin', 39.1175488, 117.1913008, <POINT (117.191 39.118)>, 13929047, 11090314]
[5, 'Guangzhou', 'Guangdong Province', 23.1301964, 113.2592945, <POINT (113.259 23.13)>, 13839429, 13965000]
[6, 'Shenzhen', 'Guangdong Province', 22.5445741, 114.0545429, <POINT (114.055 22.545)>, 12740056, 12831000]
[7, 'Chengdu', 'Sichuan', 30.6598628, 104.0633717, <POINT (104.063 30.66)>, 9412790, 9479000]
[8, 'Nanjing', 'Jiangsu', 32.0438284, 118.7788631, <POINT (118.779 32.044)>, 9320552, 9314685]
[9, 'Wuhan', 'Hubei', 30.5951051, 114.2999353, <POINT (114.3 30.595)>, 8546718, 8592000]
[10, 'Xian', 'Hebei', 38.1904931, 116.1191021, <POINT (116.119 38.19)>, 8437924, 18669]
[11, 'Hangzhou', 'Zheji

In [10]:
ans[9][1], ans[9][2], ans[9][3], ans[9][4], ans[9][5] = "Xi'an", "Shaanxi", 34.2607713, 108.9423443, Point(108.9423443, 34.2607713) # It isn't Xian, its Xi'an
ans[74][2], ans[74][3], ans[74][4], ans[74][5] = "Heilongjiang", 47.3392424, 123.9495373, Point(123.9495373, 47.3392424)    # Qiqihaer is Qiqihar
ans[173][2], ans[173][3], ans[173][4], ans[173][5] = "Xinjiang", 41.7238743, 86.173541, Point(86.173541, 41.7238743)     # Kuerle is Korla
ans[204][2], ans[204][3], ans[204][4], ans[204][5] = "Inner Mongolia", 39.6081457, 109.7763659, Point(109.7763659, 39.6081457) # Ordoss is Ordos
ans

[[1,
  'Shanghai',
  'Shanghai',
  31.2323437,
  121.4691024,
  <POINT (121.469 31.232)>,
  28242646,
  22315474],
 [2,
  'Beijing',
  'Beijing',
  39.9057136,
  116.3912972,
  <POINT (116.391 39.906)>,
  21167094,
  18960744],
 [3,
  'Chongqing',
  'Chongqing',
  29.5656729,
  106.5479189,
  <POINT (106.548 29.566)>,
  16687404,
  7457599],
 [4,
  'Tianjin',
  'Tianjin',
  39.1175488,
  117.1913008,
  <POINT (117.191 39.118)>,
  13929047,
  11090314],
 [5,
  'Guangzhou',
  'Guangdong Province',
  23.1301964,
  113.2592945,
  <POINT (113.259 23.13)>,
  13839429,
  13965000],
 [6,
  'Shenzhen',
  'Guangdong Province',
  22.5445741,
  114.0545429,
  <POINT (114.055 22.545)>,
  12740056,
  12831000],
 [7,
  'Chengdu',
  'Sichuan',
  30.6598628,
  104.0633717,
  <POINT (104.063 30.66)>,
  9412790,
  9479000],
 [8,
  'Nanjing',
  'Jiangsu',
  32.0438284,
  118.7788631,
  <POINT (118.779 32.044)>,
  9320552,
  9314685],
 [9,
  'Wuhan',
  'Hubei',
  30.5951051,
  114.2999353,
  <POINT (114.3 

In [11]:
chinese_cities = gpd.GeoDataFrame(ans, columns=header)
chinese_cities

,Rank,City,Province,Latitude,Longitude,geometry,"Population, Urban Area","Population, City"
0,1,Shanghai,Shanghai,31.232344,121.469102,POINT (121.46910 31.23234),28242646,22315474.0
1,2,Beijing,Beijing,39.905714,116.391297,POINT (116.39130 39.90571),21167094,18960744.0
2,3,Chongqing,Chongqing,29.565673,106.547919,POINT (106.54792 29.56567),16687404,7457599.0
3,4,Tianjin,Tianjin,39.117549,117.191301,POINT (117.19130 39.11755),13929047,11090314.0
4,5,Guangzhou,Guangdong Province,23.130196,113.259294,POINT (113.25929 23.13020),13839429,13965000.0
...,...,...,...,...,...,...,...,...
413,414,Hailin,Heilongjiang,44.573346,129.381575,POINT (129.38157 44.57335),332717,85578.0
414,415,Rushan,Shandong,36.913625,121.525514,POINT (121.52551 36.91362),332336,NaN
415,416,Anning,Yunnan,24.837749,102.402683,POINT (102.40268 24.83775),330956,106795.0
416,417,Rugao,Jiangsu,32.373466,120.569774,POINT (120.56977 32.37347),328336,257400.0


In [12]:
chinese_cities.to_file("cities.json", driver='GeoJSON')